# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

In [2]:
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'true',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

In [3]:
params = param1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# get model meta
model_name = params['model_name']
model_meta = mongodb_client.get_document_by_json('model', 'meta',{'model_info.model_name':model_name} )
model_meta = model_meta[0]

In [5]:
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, model_meta, db_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_testX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}
window_size: 144 nan_limit_num: 72
(6048, 24) (42, 144, 24)
(42, 1) (42, 1)


/home/leezy/anaconda3/envs/clust/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/leezy/anaconda3/envs/clust/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
# set scaler, feature list by data y flag
scaler_feature_dict = {}
if model_meta['data_y_flag']:
    scaler_feature_dict['scaler'] = scaler_y
    scaler_feature_dict['feature_list'] = params['ingestion_param_y']['feature_list']
    scaler_feature_dict['target'] = params['ingestion_param_y']['feature_list'][0]

else:
    scaler_feature_dict['scaler'] = scaler_X
    scaler_feature_dict['feature_list'] = params['ingestion_param_X']['feature_list']
    scaler_feature_dict['target'] = params['ingestion_param_y']['feature_list'][0]

In [7]:
# model test
result = ML_api.ML_test(model_meta, test_X_array, test_y_array, scaler_feature_dict)

features shape: torch.Size([42, 144, 24]) targets shape:  torch.Size([42, 1])

Start testing data



KeyError: 'scaler_param'

In [ ]:
result

In [8]:
model_meta['model_info']

{'model_purpose': 'regression',
 'data_clean_option': True,
 'model_method': 'LSTM_rg',
 'model_name': 'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 5,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'rnn_type': 'lstm',
  'input_size': 24,
  'hidden_size': 64,
  'num_layers': 2,
  'output_dim': 1,
  'dropout': 0.1,
  'bidirectional': True},
 'seq_len': 144,
 'input_size': 24,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/LSTM_rg/regression_energy_cleanLevel4_trainX_regression_LSTM_rg_/regression_energy_cleanLevel4_trainX/model.pkl'}}}